In [1]:
import os
import pandas as pd
import sys
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.lines import Line2D

from data_extraction import load_data_to_dataframe

In [2]:
# ------ Settings ------
# => These variables should be set to the desired settings!

# The files in the directory should have names of the form:
# "{dataset/other information}-factor-{downsample factor}-{any remaining information}"
directory = ''

# ---- The metrics to consider ----

# Given in the Excel file
metrics_precalc = []

# Need to be calculated here -> assumes that the Excel file contains TP, FP, TN and FN
metrics_calc = [
    'OA',
    'False Alarm',
    'Missed Alarm',
    'Recall',
    'Precision',
    'FMeasure'
]

# Filter on a thresholding algorithm
threAlg = 'OTSU'

# Variables to make plotting more convenient

datasets = {
    'OSCDDataset': 'OSCD',
    'OSCDDatasetRGBBands': 'OSCD (RGB)',
    'LEVIRCDDataset': 'LEVIR-CD'
}

algs = {
    'Change Vector Analysis': 'CVA',
    'Iterative Reweighted MAD': 'IR-MAD',
}

metrics_syn = {
    'OA' : 'Accuracy',
    'Recall' : 'Recall',
    'Precision' : 'Precision',
    'FMeasure' : 'F-score',
    'False Alarm': 'False Alarm',
    'Missed Alarm': 'Missed Alarm'
}

In [3]:
# Load the data
df = load_data_to_dataframe(directory, metrics_calc)

# Define directories
dir_dict = {
    'means_dir': os.path.join(directory, 'Plots', 'Means'),
    'heatmap_dir': os.path.join(directory, 'Plots', 'Heatmap')
}

metrics = metrics_calc + metrics_precalc

## Heatmap

Reference: https://medium.com/@szabo.bibor/how-to-create-a-seaborn-correlation-heatmap-in-python-834c0686b88e

In [4]:
def createHeatmap(data, attributes, attrs_synonyms, rows, cols, title, directory, subfolder, figname):
    data = data[attributes].rename(columns=attrs_synonyms)
    corr = data.corr().loc[rows][cols]

    sns.set_theme(font_scale=1.5, font='Times New Roman')
    plt.figure(figsize=(len(cols) * 1.4, len(rows) * 1.2))
    sns.heatmap(corr, vmin = -1, vmax = 1, center = 0, annot=True, fmt='.3f', linewidths=2.2, cmap = 'coolwarm')
    # hm.set_title(title, fontdict={'fontsize':18, 'weight':'bold'}, pad=14)
    
    folder = os.path.join(directory, subfolder)
    if not os.path.exists(folder):
        os.makedirs(folder)

    plt.xticks(rotation=0)
    plt.yticks(rotation=0)
    plt.savefig(os.path.join(folder, figname + '.png'), bbox_inches='tight')
    plt.close()

In [5]:
datasets = {
    'OSCDDataset': 'OSCD',
    'OSCDDatasetRGBBands': 'OSCD (RGB)',
    'LEVIRCDDataset': 'LEVIR-CD'
}
attrs = [#'Size', #'refWidth', 'refHeight',
          'scaledRes', 'OA', 'Precision', 'Recall', 'FMeasure','False Alarm', 'Missed Alarm']
attrs_syn = {
    #'refWidth': 'Width\n[px]', 
    #'refHeight': 'Height\n[px]', 
    #'Size': 'Size\n[px × px]', 
    'scaledRes': 'Resolution\n[m/px]', 
    'OA': 'Accuracy',
    'Precision': 'Precision',
    'Recall': 'Recall',
    'FMeasure': 'F-score',
    'False Alarm': 'False\nAlarm',
    'Missed Alarm': 'Missed\nAlarm'
}
cols = [attrs_syn[x] for x in ['OA', 'False Alarm', 'Missed Alarm', 'Precision', 'Recall', 'FMeasure']]
rows = [attrs_syn[x] for x in ['scaledRes']]#, 'Size']]#, 'refWidth', 'refHeight']]

In [6]:
plt.ioff()

for alg in algs.keys():
    createHeatmap(
        df.loc[(df['name'] == alg) & (df['threAlg'] == threAlg)],
        attrs, attrs_syn,
        rows, cols,
        f'{alg}',
        dir_dict['heatmap_dir'],
        alg,
        f'{algs[alg]}-{threAlg}-all'
    )
    
    for ds in datasets.keys():
        createHeatmap(
            df.loc[(df['name'] == alg) & (df['threAlg'] == threAlg) & (df['dataset'] == ds)],
            attrs, attrs_syn,
            rows, cols,
            f'{alg} for {datasets[ds]}',
            dir_dict['heatmap_dir'],
            alg,
            f'{algs[alg]}-{threAlg}-{ds}'
        )

In [7]:
levir_imgs = ['test_89', 'test_77', 'test_38', 'test_39', 'test_59']
oscd_imgs = ['milano-cm', 'valencia-cm', 'dubai-cm']
imgs = levir_imgs + oscd_imgs

for img in imgs:
    for alg in algs.keys():
        createHeatmap(
            df.loc[(df['name'] == alg) & (df['threAlg'] == threAlg) & (df['refName'] == img)],
            attrs, attrs_syn,
            rows, cols,
            f'{alg} {img}',
            dir_dict['heatmap_dir'],
            img,
            f'{algs[alg]}-{threAlg}-{img}'
        )

## Means

In [8]:
plt.rc_context({'plot.edgecolors': 'red'})

def plotMeanDiffAlgsBetter(
        data, algs, datasets,
        x_attr, x_label,
        y_attr, y_label, ylim,
        title, legend, styles,
        directory, subfolder, figname):
    
    _, ax = plt.subplots(edgecolor='red')
    
    for alg in algs.keys():
        for ds in datasets.keys():
            style = styles[algs[alg]][ds]

            data.loc[(data['name'] == alg) & (data['dataset'] == ds)] \
                [[x_attr, y_attr]].groupby(x_attr).mean() \
                    .plot.line(ax=ax, style=style[1], color=style[0])
    
    plt.ylim(ylim[0], ylim[1])
    plt.xlabel(x_label, fontsize=14)
    plt.ylabel(y_label, fontsize=14)
    plt.title(title, fontsize=18, pad=9, weight='bold')
    plt.legend(handles=legend, fontsize=14)

    ax.grid(color = 'gainsboro', linestyle = '--', linewidth = 1.3)
    ax.set_facecolor('white')
    for spine in ax.spines.values():
        spine.set_linewidth(1)
        spine.set_color('black')

    folder = os.path.join(directory, subfolder)
    if not os.path.exists(folder):
        os.makedirs(folder)
    plt.savefig(os.path.join(folder, figname), bbox_inches='tight')
    plt.close()

# Style settings
plt.rcParams["font.family"] = "Times New Roman"
colors = ['#8a8a8a', '#2e2e2e']
styles = {
    'CVA': {
        'OSCDDataset': [colors[0], ':o'],
        'OSCDDatasetRGBBands': [colors[0], '-o'],
        'LEVIRCDDataset': [colors[0], '-o']
    },
    'IR-MAD': {
        'OSCDDataset': [colors[1], ':o'],
        'OSCDDatasetRGBBands': [colors[1], '-o'],
        'LEVIRCDDataset': [colors[1], '-o']
    }
}

In [9]:
# Only three bands
datasets = {
    'OSCDDatasetRGBBands': 'OSCD (RGB)',
    'LEVIRCDDataset': 'LEVIR-CD'
}
legend = [
    Line2D([0], [0], color=colors[0], marker='o',lw=2, label='CVA'),
    Line2D([0], [0], color=colors[1], marker='o', lw=2, label='IR-MAD')]
subfolder = 'OSCD only RGB bands'

plt.ioff()

for metric in metrics_calc:
    plotMeanDiffAlgsBetter(
        df.loc[(df['threAlg'] == threAlg)], algs, datasets,
        'scaledRes', 'Resolution [m/px]',
        metric, metrics_syn[metric], (0,1),
        metrics_syn[metric], legend, styles,
        dir_dict['means_dir'], subfolder, f'{metric}-{threAlg}-resolution'
    )

In [10]:
# Both OSCD in one plot
datasets = {
    'OSCDDataset': 'OSCD',
    'OSCDDatasetRGBBands': 'OSCD (RGB)'
}
legend = [
    Line2D([0], [0], color=colors[0], marker='o', lw=2, label='CVA - RBG'),
    Line2D([0], [0], color=colors[0], marker='o', ls=':', lw=2, label='CVA - all bands'),
    Line2D([0], [0], color=colors[1], marker='o', lw=2, label='IR-MAD - RGB'),
    Line2D([0], [0], color=colors[1], marker='o', ls=':', lw=2, label='IR-MAD - all bands')]
subfolder = 'Only OSCD comparison'

plt.ioff()

for metric in metrics_calc:
    plotMeanDiffAlgsBetter(
        df.loc[(df['threAlg'] == threAlg)], algs, datasets,
        'scaledRes', 'Resolution [m/px]',
        metric, metrics_syn[metric], (0,1),
        metrics_syn[metric], legend, styles,
        dir_dict['means_dir'], subfolder, f'{metrics_syn[metric]}-{threAlg}-resolution'
    )

In [11]:
# Specific LEVIR images - version 16.06
datasets = {
    'LEVIRCDDataset': 'LEVIR-CD'
}
legend = [
    Line2D([0], [0], color=colors[0], marker='o',lw=2, label='CVA'),
    Line2D([0], [0], color=colors[1], marker='o', lw=2, label='IR-MAD')]
subfolder = 'LEVIR-CD images'

plt.ioff()

levir_imgs = ['test_89', 'test_104', 'test_16', 'test_102', 'test_77', 'test_116', 'test_112', 'test_109', 'test_45', 'test_38', 'test_40', 'test_39']
imgs = levir_imgs

for metric in metrics_calc:
    for img in imgs:
        plotMeanDiffAlgsBetter(
            df.loc[(df['threAlg'] == threAlg) & (df['refName'] == img)], algs, datasets,
            'scaledRes', 'Resolution [m/px]',
            metric, metrics_syn[metric], (0,1),
            metrics_syn[metric] + ' for ' + img, legend, styles,
            dir_dict['means_dir'], os.path.join(subfolder, img), f'{metric}-{threAlg}-resolution-{img}'
        )